In [1]:
from dask.distributed import Client, LocalCluster

from matplotlib import pyplot as plt
from itkwidgets import view

import dask.array.image
import numpy as np
import itk
import dask.array as da
import dask

import os
import time

## Specify the distributed computing resources

Dask supports running on HPC MPI, HPC job schedulers, cloud-based clusters, or a set of systems connected over SSH.

As a first step, we specify how to connect to our distributed computing resources.

In this example, we will test with a client that just uses the multiple cores on our local systems.

In [2]:
# Option 1: local cluster

# local_cluster = LocalCluster(n_workers=8, processes=False, memory_limit='4G')
# client = Client(local_cluster)

In [3]:
# Option 2: NERSC Cori Slurm dask-mpi cluster
#
# See SC20_pyHPC/nersc/README.md

scheduler_file = os.path.join(os.environ["SCRATCH"], "scheduler.json")
dask.config.config["distributed"]["dashboard"]["link"] = "{JUPYTERHUB_SERVICE_PREFIX}proxy/{host}:{port}/status"

client = Client(scheduler_file=scheduler_file)

In [4]:
client

Client Scheduler: tcp://10.128.0.20:33623 Dashboard: /user/thewtex/cori-shared-node-cpu/proxy/10.128.0.20:33473/status,Cluster Workers: 99 Cores: 99 Memory: 0 B


In [5]:
# An example image processing pipeline with a parameter
def my_processing_pipeline(image_chunk, radius=2):
    import itk
    
    denoised = itk.median_image_filter(image_chunk, radius=radius)
    return denoised

In [6]:
input_filepath = '../data/bead_pack.tif'

In [7]:
image = itk.imread(input_filepath)
image = np.asarray(image)

## Option 0: Non-distributed

In [8]:
start = time.time()

denoised = my_processing_pipeline(image, radius=2)

elapsed = time.time() - start
print(elapsed, 'seconds')

3.7748758792877197 seconds


## Option 1: Dask Client.submit

### Submit the processing pipeline and image as a task

In [9]:
start = time.time()

denoised = client.submit(my_processing_pipeline, np.asarray(image)).result()

elapsed = time.time() - start
print(elapsed, 'seconds')

Large object of size 8.00 MB detected in task graph: 
  (array([[[112, 107, 112, ..., 113, 117, 117],
     ...  dtype=uint8),)
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good


3.8913466930389404 seconds


In [10]:
# Chunk the data into a Dask array.
chunks = chunks=(5, 200, 200) # 40 blocks

chunked_data = da.from_array(image, chunks=chunks)
chunked_data

,Array,Chunk
Bytes,8.00 MB,200.00 kB
Shape,"(200, 200, 200)","(5, 200, 200)"
Count,41 Tasks,40 Chunks
Type,uint8,numpy.ndarray


### Scatter the data across the clients

In [11]:
# Scatter the data across the clients
scattered_data = client.scatter(chunked_data, broadcast=True)

start = time.time()

denoised = client.submit(my_processing_pipeline, scattered_data).result()

elapsed = time.time() - start
print(elapsed, 'seconds')

4.177775144577026 seconds


distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError


## Option 2: Dask Array map_blocks

In [12]:
start = time.time()

denoised = da.map_blocks(my_processing_pipeline,
              chunked_data,
              radius=2,
              dtype=chunked_data.dtype)
denoised = denoised.compute()

elapsed = time.time() - start
print(elapsed, 'seconds')

7.0909483432769775 seconds


In [13]:
view(denoised)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageUC3; pr…

## Option 3: Dask Array map_overlap

Extend the processed region, then trim the result.

In [14]:
start = time.time()

denoised = da.map_overlap(my_processing_pipeline,
              chunked_data,
              depth=2,
              trim=True,
              radius=2,
              dtype=chunked_data.dtype)
denoised = denoised.compute()

elapsed = time.time() - start
print(elapsed, 'seconds')

8.25371789932251 seconds


## Option 4: Xarray DataArray map_blocks

In [15]:
image = itk.imread(input_filepath)
da = itk.xarray_from_image(image)
da

<xarray.DataArray (z: 200, y: 200, x: 200)>
array([[[112, 107, 112, ..., 113, 117, 117],
        [109, 107, 102, ..., 110, 111, 112],
        [110, 109, 109, ..., 115, 108, 114],
        ...,
        [195, 200, 201, ..., 195, 200, 204],
        [197, 203, 197, ..., 207, 209, 210],
        [200, 197, 196, ..., 203, 205, 208]],

       [[107, 118, 117, ..., 113, 115, 125],
        [106, 116, 104, ..., 114, 107, 118],
        [110, 107, 109, ..., 112, 115, 117],
        ...,
        [202, 200, 200, ..., 209, 210, 213],
        [207, 197, 209, ..., 201, 214, 203],
        [195, 200, 204, ..., 207, 204, 208]],

       [[115, 111, 116, ..., 107, 116, 113],
        [105, 108, 107, ..., 109, 111, 120],
        [111, 110, 109, ..., 116, 113, 145],
        ...,
...
        ...,
        [198, 202, 200, ..., 204, 200, 204],
        [194, 204, 198, ..., 199, 207, 200],
        [201, 206, 204, ..., 201, 203, 204]],

       [[113, 108, 111, ..., 212, 198, 202],
        [108, 105, 109, ..., 206, 197, 202],
        [114, 107, 104, ..., 204, 203, 201],
        ...,
        [197, 206, 199, ..., 151, 160, 170],
        [199, 195, 198, ..., 200, 206, 202],
        [197, 197, 199, ..., 210, 202, 208]],

       [[115, 109, 109, ..., 206, 203, 204],
        [108, 103, 110, ..., 194, 201, 204],
        [108, 110, 112, ..., 207, 212, 199],
        ...,
        [203, 200, 203, ..., 108, 111, 116],
        [200, 199, 196, ..., 119, 133, 135],
        [200, 203, 199, ..., 146, 152, 152]]], dtype=uint8)
Coordinates:
  * x        (x) float64 0.0 1.0 2.0 3.0 4.0 ... 195.0 196.0 197.0 198.0 199.0
  * y        (y) float64 0.0 1.0 2.0 3.0 4.0 ... 195.0 196.0 197.0 198.0 199.0
  * z        (z) float64 0.0 1.0 2.0 3.0 4.0 ... 195.0 196.0 197.0 198.0 199.0
Attributes:
    direction:  [[1. 0. 0.]\n [0. 1. 0.]\n [0. 0. 1.]]

In [16]:
da = da.chunk(chunks)

In [17]:
start = time.time()

denoised = da.map_blocks(my_processing_pipeline,
              kwargs = { 'radius': 2 } )
denoised = denoised.compute()

elapsed = time.time() - start
print(elapsed, 'seconds')

5.544623851776123 seconds
